### This notebook helps to do 3 things:
- Paso 1: Excecute the docXChain to process a pdf into JSON (semi-parametrizado)
- Paso 2: Convert Json into DataFrame
- Paso 3: DataFrame to Document
- Paso 4: Crea la nueva función pdf_process.py

In [3]:
#- Paso 1: Exe DocXChain
import subprocess

path_to_docXchain = "/home/camilo/Documents/01-tutoriales/docxchain/Applications/DocXchain/"
path_to_pdfs = "/home/camilo/Documents/by_species/"
species_folder = "Melipona_beecheii/"#"Leptonycteris/"
arg1_fun = "whole_pdf_conversion"
file_name = "Castillo-Aguilar et al., 2015"
arg2_input = file_name+".pdf"
arg3_output = file_name+".json"
#Hensley_test.json"

args = ["python", path_to_docXchain+"example.py", arg1_fun, path_to_pdfs+species_folder+arg2_input, path_to_docXchain+"output/"+arg3_output]
#args = ["python", "-c", f"import example; example.whole_pdf_conversion('{path_to_pdfs+species_folder+arg2_input}', '{path_to_docXchain+'output/'+arg3_output}')"]
result = subprocess.run(args, capture_output=True, text=True)

print(result.stdout)
print(result.stderr)

2024-09-03 23:50:52
Started!
Fix size testing.
training chunk_sizes: [32]
The output will be saved to  /home/camilo/Documents/01-tutoriales/docxchain/Applications/DocXchain/modules/../../../DocumentUnderstanding/DocXLayout/../../exp/ctdet_subfield/default
heads {'hm': 11, 'cls': 4, 'ftype': 3, 'wh': 8, 'hm_sub': 2, 'wh_sub': 8, 'reg': 2, 'reg_sub': 2}
Namespace(task='ctdet_subfield', dataset='huntie', test=False, data_src='default', exp_id='default', vis_corner=0, convert_onnx=0, onnx_path='auto', debug=0, load_model='/home/DocXLayout_231012.pth', resume=False, gpus=[0], num_workers=16, not_cuda_benchmark=False, seed=317, print_iter=0, hide_data_time=False, save_all=False, metric='loss', vis_thresh=0.3, nms_thresh=0.3, corner_thresh=0.3, debugger_theme='white', arch='dlav0subfield_34', head_conv=256, down_ratio=4, input_res=768, input_h=768, input_w=768, lr=0.000125, lr_step=[80], NotFixList='', num_epochs=90, batch_size=32, master_batch_size=32, num_iters=-1, val_intervals=5, trainval

### - Paso 2: Convert Json into csv

In [12]:
import pandas as pd
import json
from pathlib import Path
file_name = path_to_docXchain+"output/"+arg3_output
#file_name = "test_no-image.json"

with open(file_name, 'r') as f:
    data = json.load(f)
print(data[0].keys())

dict_keys(['page', 'information'])


In [5]:
df = pd.DataFrame(data)
new_df = df
# you could try 
# col = df.map(type).astype(str).eq("<class 'list'>").all().idxmax()
# 'region_poly' would satisfy that condition but doesn't need to be expoloted. Besides, the structure should be stable
# Hence: column_names hard-code
cols = ['information','text_list']

for col in cols:
    print(col)
    new_df = new_df.explode(col).reset_index(drop=True)
    new_df = new_df.drop(columns=[col]).join(new_df[col].apply(pd.Series), rsuffix=f".{col}")

new_df

information
text_list


,page,category_index,category_name,region_poly,position,content,0
0,0,0,title,"[177, 633, 293, 633, 293, 655, 177, 655]","[180, 632, 295, 632, 295, 655, 180, 654]",[RESUMEN],NaN
1,0,0,title,"[352, 113, 921, 113, 921, 257, 352, 257]","[345, 112, 923, 112, 923, 157, 345, 157]",[PRODUCCION DE PLANTA],NaN
2,0,0,title,"[352, 113, 921, 113, 921, 257, 352, 257]","[397, 162, 871, 161, 871, 204, 397, 205]",[DE CHILE HABANERO],NaN
3,0,0,title,"[352, 113, 921, 113, 921, 257, 352, 257]","[353, 212, 914, 210, 914, 258, 353, 260]",[(Capsicum chinense Jacq.)],NaN
4,0,0,title,"[176, 898, 302, 898, 302, 918, 176, 918]","[178, 895, 300, 895, 300, 917, 178, 917]",[ABSTRACT],NaN
...,...,...,...,...,...,...,...
450,5,6,footer,"[44, 1592, 219, 1592, 219, 1615, 44, 1615]","[154, 1593, 218, 1593, 218, 1607, 154, 1607]",[AGRO],NaN
451,5,6,footer,"[44, 1592, 219, 1592, 219, 1615, 44, 1615]","[42, 1592, 65, 1592, 65, 1612, 42, 1612]",[78],NaN
452,5,6,footer,"[44, 1592, 219, 1592, 219, 1615, 44, 1615]","[75, 1591, 106, 1592, 106, 1612, 75, 1612]",[11],NaN
453,5,6,footer,"[44, 1592, 219, 1592, 219, 1615, 44, 1615]","[102, 1591, 143, 1591, 143, 1612, 102, 1612]",[AP],NaN


In [25]:
file_outout_name = file_name.split(".")[0]+".csv"
print(file_outout_name)
new_df.to_csv(file_outout_name, header=True, index=False)

/home/camilo/Documents/01-tutoriales/docxchain/Applications/DocXchain/output/Gómez-Ruiz et al.csv


### - Paso 3: DataFrame to Document

In [120]:
# almost every error comes from not having the right data_type in column content. 
# It's very convenient to filter out problematic and non relevant categories from the beguinning
new_df.category_name.unique()
#new_df[new_df.category_name == 'figure']
#new_df[new_df.category_name == 'table']
#clean_df['content'] = new_df['content'].apply(lambda x: x[0] if isinstance(x,list) else '')
#clean_df['content']

array(['title', 'plain text', 'header', 'footnote', 'footer', 'figure',
       'page number', 'figure caption', 'table', 'table caption'],
      dtype=object)

In [6]:
df = pd.DataFrame(data)
new_df = df
# you could try 
# col = df.map(type).astype(str).eq("<class 'list'>").all().idxmax()
# 'region_poly' would satisfy that condition but doesn't need to be expoloted. Besides, the structure should be stable
# Hence: column_names hard-code
cols = ['information','text_list']

for col in cols:
    print(col)
    new_df = new_df.explode(col).reset_index(drop=True)
    new_df = new_df.drop(columns=[col]).join(new_df[col].apply(pd.Series), rsuffix=f".{col}")

new_df['content'] = new_df['content'].apply(lambda x: x[0] if isinstance(x,list) else '')
new_df['region_poly'] = new_df['region_poly'].apply(lambda x: tuple(x)) # lists are not mutable 
new_df['content_type'] = new_df['content'].apply(lambda x: type(x).__name__)

df_text = new_df[new_df.category_name == 'plain text']


information
text_list


In [7]:
aggr_level = ['page', 'region_poly']
grouped_df = df_text.groupby(aggr_level)

# Concatenate the content of the 'content' column with spaces
test = df_text.groupby(aggr_level)['content'].apply(lambda x: ' '.join(x)).reset_index()
test

,page,region_poly,content
0,0,"(177, 669, 1094, 669, 1094, 805, 177, 805)",La produccion de planta de chile habanero (Cap...
1,0,"(177, 845, 492, 845, 492, 862, 177, 862)","Palabras clave: vivero, Campeche. chiles"
2,0,"(177, 1108, 474, 1108, 474, 1127, 177, 1127)","Keywords: Nursery, Campeche, chiles"
3,0,"(191, 933, 1082, 933, 1082, 1064, 191, 1064)",Habanero pepper plant production (Capsicum chi...
4,0,"(211, 417, 1063, 417, 1063, 437, 211, 437)","Castillo-Aguilar,C. de la C.;Quej-Ch,iV.;Coh-M..."
5,0,"(264, 499, 725, 499, 725, 519, 264, 519)",municipio de Champoton. Campeche. C. P. 24450....
6,0,"(266, 523, 567, 523, 567, 542, 266, 542)",*Autor responsable: cccaocolpos.mx
7,0,"(269, 473, 994, 473, 994, 494, 269, 494)","'Campus Campeche, Coleqio de Postqraduados. Ca..."
8,0,"(355, 312, 913, 312, 913, 375, 355, 375)",HABANERO PEPPER PLANT PRODUCTION (Capsicum chi...
9,0,"(1055, 1592, 1223, 1592, 1223, 1615, 1055, 1615)",AGRO AP | 73


In [27]:
from langchain_core.documents import Document
my_docs = [Document(
            page_content = row.content, 
            metadata = dict(page = row.page,
                        source = Path(path_to_pdfs).name,
                        species_folder = species_folder)
            )
            for _,row in test.iterrows()
        ]

In [28]:
my_docs

[Document(metadata={'page': 0, 'source': 'by_species', 'species_folder': 'Melipona_beecheii/'}, page_content='La produccion de planta de chile habanero (Capsicum chinense Jacq) es una actividad crucial en la produccián exitosa de esta especie. donde confluyen tanto las habilidades de los productores como el conocimiento tecnico. La informacion presentada en el presente trabajo considera en su mayoria el conocimiento adquirido por tecnicos y productores en el estado de Campeche. Dentro de las actividades mas importantes en la produccion de planta que se describen estan el uso de variedades. siembra y manejo de la planta en el vivero, informacion que sera de uilidad a yiveristas y nroductoree'),
 Document(metadata={'page': 0, 'source': 'by_species', 'species_folder': 'Melipona_beecheii/'}, page_content='Palabras clave: vivero, Campeche. chiles'),
 Document(metadata={'page': 0, 'source': 'by_species', 'species_folder': 'Melipona_beecheii/'}, page_content='Keywords: Nursery, Campeche, chil

In [153]:
from langchain_core.documents import Document
from pymupdf4llm import to_markdown
from pathlib import Path
import os
import time
import logging
#from find_info_app.logging import get_logger


In [154]:
root_string = '/home/camilo/Documents/by_species/'
root_folder = Path(root_string)
#This can be a file in the root dir to read from
processed_list=['Lerptonycteris yerbabuenae', 'Leptonycteris nivalis', 'Melipona beecheii']
species_subfolders = os.listdir(root_folder)
pending_process_species = list(set(species_subfolders)-set(processed_list))
pending_process_species 

['test species2', 'test species']

In [155]:
pdf_data = []
for species_folder in pending_process_species:
    pdf_data.append(dict(species_folder_path = Path(root_folder,species_folder),
                    file_names = os.listdir(Path(root_folder,species_folder,species_folder+'_bibliografía'))
    )
    )
        
for item in pdf_data:
   species_folder_path = item['species_folder_path']
   for file_name in item['file_names']:
       print(file_name)

Narciso-Reyes, 2003.pdf
Genaro, 2018.pdf
Paxton et al., 1999.pdf
Simmons, 2005.pdf
Arita, 1991.pdf
Pourshoushtari, 2019.pdf


In [157]:
file_ = Path(species_folder_path,species_folder_path.name+'_bibliografía',file_name)
pages = to_markdown(file_, page_chunks=True)

In [158]:
pages[0].get("metadata")

{'format': 'PDF 1.5',
 'title': '',
 'author': 'Roxanne',
 'subject': '',
 'keywords': '',
 'creator': 'Acrobat PDFMaker 15 for Word',
 'producer': 'Adobe PDF Library 15.0',
 'creationDate': "D:20190430135620-05'00'",
 'modDate': "D:20190430140340-05'00'",
 'trapped': '',
 'encryption': None,
 'file_path': '/home/camilo/Documents/by_species/test species/test species_bibliografía/Pourshoushtari, 2019.pdf',
 'page_count': 73,
 'page': 1}

In [159]:
LONG_DOCUMENT_SIZE = 50
def is_long_doc(metadata: dict) -> bool:
    page_count = metadata.get("page_count", 0)
    page = metadata.get("page")
    filename = metadata.get("source")
    if page_count > LONG_DOCUMENT_SIZE:
        if page == 1:
            
            logging.warning(
                f"Document {filename} skipped, it has {page_count} pages, but 50 is the limit"
            )
        return True

    return False

In [161]:
path_to_docXchain = "/home/camilo/Documents/01-tutoriales/docxchain/Applications/DocXchain/"
arg1_fun = "whole_pdf_conversion"

for item in pdf_data:
    
    species_folder_path = item['species_folder_path']
    output_folder = Path(species_folder_path,'output')

    if not os.path.exists(output_folder):
            print("Creating Folder: {output_folder}")
            os.mkdir(output_folder)
    
    for file_name in item['file_names']: 

        start_time = time.time()
        print(f'Start OCR process for: {Path(species_folder_path,file_name)}')
        arg1_fun = "whole_pdf_conversion"
        arg2_input = Path(species_folder_path,species_folder_path.name+'_bibliografía',file_name)
        arg3_output  = Path(output_folder,file_name+".json")
        
        md_metadata = to_markdown(arg2_input, page_chunks=True)[0].get("metadata")
        
        if is_long_doc(md_metadata):
            continue

        if os.path.exists(arg3_output):
            print(f"Skipping {arg3_output.name} - file already exists.")
            break
        
        args = ["python", path_to_docXchain+"example.py", arg1_fun, arg2_input, arg3_output]
        
        result = subprocess.run(args, capture_output=True, text=True)
        end_time = time.time()
        elapsed_time = end_time - start_time
        logging.info(f"Processed {file_name} in {elapsed_time:.2f} seconds")
        print(result.stdout)
        print(result.stderr)


Start OCR process for: /home/camilo/Documents/by_species/test species2/Narciso-Reyes, 2003.pdf
Skipping Narciso-Reyes, 2003.pdf.json - file already exists.
Start OCR process for: /home/camilo/Documents/by_species/test species/Simmons, 2005.pdf


Start OCR process for: /home/camilo/Documents/by_species/test species/Arita, 1991.pdf
Skipping Arita, 1991.pdf.json - file already exists.


In [ ]:

import subprocess

path_to_docXchain = "/home/camilo/Documents/01-tutoriales/docxchain/Applications/DocXchain/"
path_to_pdfs = "/home/camilo/Documents/by_species/"
species_folder = "Melipona_beecheii/"#"Leptonycteris/"
arg1_fun = "whole_pdf_conversion"
file_name = "Castillo-Aguilar et al., 2015"
arg2_input = file_name+".pdf"
arg3_output = file_name+".json"

#Hensley_test.json"

args = ["python", path_to_docXchain+"example.py", arg1_fun, path_to_pdfs+species_folder+arg2_input, root_folder+species_folder+"output/"+arg3_output]
#args = ["python", "-c", f"import example; example.whole_pdf_conversion('{path_to_pdfs+species_folder+arg2_input}', '{path_to_docXchain+'output/'+arg3_output}')"]
result = subprocess.run(args, capture_output=True, text=True)

print(result.stdout)
print(result.stderr)